Übung:  09-2 Ingress
-------------------

![](demo/images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Statt des Apache-Webservers, der als [Reverse Proxy](https://github.com/ewolff/microservice-kubernetes/blob/master/microservice-kubernetes-demo/apache/000-default.conf) konfiguriert ist, wird die Kubernetes Ressource Ingress verwendet.

In [ ]:
%%bash
kubectl create namespace ms-kubernetes
# ! kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml (obsolet!)
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

Nach dem Starten der Services erstellen wir die Ingress Ressourcen:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /\$2
  name: order
  namespace: ms-kubernetes 
  labels:
    app: order
spec:
  rules:
  - http:
      paths:
      - path: /order/
        pathType: Prefix
        backend:
          service:
            name: order
            port:
              number: 8080
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: catalog
  namespace: ms-kubernetes 
  labels:
    app: catalog
spec:
  rules:
  - http:
      paths:
      - path: /catalog
        pathType: Prefix
        backend:
          service:
            name: catalog
            port:
              number: 8080
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: customer 
  namespace: ms-kubernetes 
  labels:
    app: customer 
spec:
  rules:
  - http:
      paths:
      - path: /customer
        pathType: Prefix
        backend:
          service:
            name: customer
            port:
              number: 8080  
%EOF%


Überprüfen der erstellen Ressourcen

In [ ]:
! kubectl get all,ingress -n ms-kubernetes

Wir kontrollieren die korrekte Funktionsweise mittels `curl` (Window `Invoke-WebRequest`).

In [ ]:
%%bash
export SERVER=https://$(cat ~/work/server-ip)
echo "Kunden ${SERVER}/customer"
curl -k ${SERVER}/customer
echo "Produkte ${SERVER}/catalog"
curl -k ${SERVER}/catalog
# echo "Bestellung ${SERVER}/order"
# curl -k ${SERVER}/order/<Order-id>

***
### Ingress Service (nginx Server)

In der aktuellen Umgebung übernimmt ein nginx Server die Ingress Funktionalität. Dieser Server läuft als Pods im Namespace ingress-nginx.

Von dem nginx Server können wir uns die Konfigurationsdatei ausgeben:

In [ ]:
! kubectl exec daemonset/nginx-ingress-microk8s-controller -n ingress -- cat /etc/nginx/nginx.conf | grep location

Zum Testen kann der `kubectl apply -f -` welche die Ingress Ressourcen anlegt, durch `kubectl delete -f -` ersetzt werden und dann der obige Befehl wieder ausgeführt werden.

Dann sollten die `location` Einträge für `customer`, `catalog` und `order` nicht mehr vorhanden sein.

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-kubernetes

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten


Was ein Ingress?
<details><summary>Antwort</summary><p>    
    Ein API-Objekt, das den externen Zugriff auf die Dienste in einem Cluster verwaltet, in der Regel mittels HTTP. 
    Grob entspricht der Ingress Dienst dem Reverse Proxy Muster. 
</p></details>

---

Was für eine Netzwerkstruktur verwendet Kubernetes?
<details><summary>Antwort</summary><p>    
     Kubernetes verwendet im Unterschied zu Docker eine flache Netzwerkstruktur. 
* Jeder Container kann mit jedem anderen ohne NAT kommunizieren.
* Alle Kubernetes Nodes können mit allen Containern (und in die andere Richtung) ohne NAT kommunizieren.
* Die IP, die ein Container von sich selbst sieht, ist auch die, die jeder andere Node oder Container im Netz von ihm sieht.
</p></details>

---